# Get Eva's Instagram details

In [1]:
import json
import pandas as pd

In [2]:
# Run this with real username, password to scrape Insta objects
# !instagram-scraper evahustiles -u USERNAME -p PASSWORD --media-metadata

In [3]:
with open('evahustiles/evahustiles.json') as f:
  data = json.load(f)

In [4]:
insta_data = data['GraphImages']

In [5]:
def simple_dict(inst_dict):
  to_return = {}
  to_return['datetime'] = inst_dict['taken_at_timestamp']
  to_return['likes'] = inst_dict['edge_media_preview_like']['count']
  to_return['id'] = inst_dict['id']
  return to_return

In [6]:
data = [simple_dict(v) for v in insta_data]

In [7]:
src = pd.DataFrame(data)

In [8]:
src.head()

,datetime,likes,id
0,1639965815,7,2732552546491095839
1,1639332235,14,2727238920394593041
2,1638728978,18,2722178432111568080
3,1638646923,23,2721490102072084593
4,1637897085,17,2715200003060964019


### Process dates

In [9]:
src["datetime"] = pd.to_datetime(src["datetime"], unit='s')

In [10]:
src['date'] = src['datetime'].dt.tz_localize("GMT").dt.tz_convert('America/Los_Angeles').dt.date
src['time'] = src['datetime'].dt.tz_localize("GMT").dt.tz_convert('America/Los_Angeles').dt.time

In [11]:
src["weekday"] = pd.to_datetime(src["datetime"]).dt.day_name()
src["year"] = pd.to_datetime(src["datetime"]).dt.year
src["month"] = pd.to_datetime(src["datetime"]).dt.month_name()
src["weekend"] = pd.to_datetime(src["datetime"]).dt.day_name().isin(["Saturday", "Sunday"])

In [12]:
src.head()

,datetime,likes,id,date,time,weekday,year,month,weekend
0,2021-12-20 02:03:35,7,2732552546491095839,2021-12-19,18:03:35,Monday,2021,December,False
1,2021-12-12 18:03:55,14,2727238920394593041,2021-12-12,10:03:55,Sunday,2021,December,True
2,2021-12-05 18:29:38,18,2722178432111568080,2021-12-05,10:29:38,Sunday,2021,December,True
3,2021-12-04 19:42:03,23,2721490102072084593,2021-12-04,11:42:03,Saturday,2021,December,True
4,2021-11-26 03:24:45,17,2715200003060964019,2021-11-25,19:24:45,Friday,2021,November,False


In [13]:
df = src.copy()

In [14]:
df.value_counts('weekend', normalize=True)

weekend
False    0.64
True     0.36
dtype: float64

In [15]:
df.value_counts('weekday', normalize=True)

weekday
Monday       0.22
Sunday       0.20
Friday       0.16
Saturday     0.16
Thursday     0.12
Tuesday      0.08
Wednesday    0.06
dtype: float64

In [16]:
df.to_csv('data/instagram_metadata.csv', index=False)